### 贝叶斯优化优势

1、可以考虑先验信息，来不断更新先验；

2、相比网格搜索，迭代次数少速度快；

3、针对非凸问题依然稳健；


-----
### 安装

-----
###   Tutorial Code
https://github.com/fmfn/BayesianOptimization


In [1]:
def black_box_function(x, y):
    """Function with unknown internals we wish to maximize.

    This is just serving as an example, for all intents and
    purposes think of the internals of this function, i.e.: the process
    which generates its output values, as unknown.
    """
    return -x ** 2 - (y - 1) ** 2 + 1

In [2]:
from bayes_opt import BayesianOptimization

# Bounded region of parameter space
pbounds = {'x': (2, 4), 'y': (-3, 3)}

optimizer = BayesianOptimization(
    f=black_box_function,
    pbounds=pbounds,
    random_state=1,
)

In [ ]:
optimizer.maximize(
    init_points=2, #起始随机测试次数
    n_iter=3,      #优化迭代次数
)
print(optimizer.max)
#  optimizer.res[0]

|   iter    |  target   |     x     |     y     |
-------------------------------------------------
|  1        | -7.135    |  2.834    |  1.322    |
|  2        | -7.78     |  2.0      | -1.186    |
|  3        | -19.0     |  4.0      |  3.0      |
|  4        | -16.3     |  2.378    | -2.413    |
|  5        | -4.441    |  2.105    | -0.005822 |
{'params': {'y': -0.005822117636089974, 'x': 2.104665051994087}, 'target': -4.441293113411222}


In [ ]:
#   调整参数边界
optimizer.set_bounds(new_bounds={"x": (-2, 3)})

optimizer.maximize(
    init_points=0,
    n_iter=5,
)

|   iter    |  target   |     x     |     y     |
-------------------------------------------------
|  6        | -5.145    |  2.115    | -0.2924   |


In [40]:
#   指定点进行贝叶斯优化
optimizer.probe(
    params={"x": 0.5, "y": 0.7},
    lazy=True,
)

optimizer.probe(
    params=[-0.3, 0.1],
    lazy=True,
)

# Will probe only the two points specified above
optimizer.maximize(init_points=0, n_iter=0)

|   iter    |  target   |     x     |     y     |
-------------------------------------------------
|  11       |  0.66     |  0.5      |  0.7      |
|  12       |  0.1      | -0.3      |  0.1      |


In [ ]:
#   保存优化过程
from bayes_opt.observer import JSONLogger
from bayes_opt.event import Events

logger = JSONLogger(path="./bayesopt_model.json")
optimizer.subscribe(Events.OPTMIZATION_STEP, logger)

# Results will be saved in ./logs.json
optimizer.maximize(
    init_points=2,
    n_iter=3,
)

In [ ]:
#   读取优化过程
from bayes_opt.util import load_logs


new_optimizer = BayesianOptimization(
    f=black_box_function,
    pbounds={"x": (-2, 2), "y": (-2, 2)},
    verbose=2,
    random_state=7,
)

# New optimizer is loaded with previously seen points
load_logs(new_optimizer, logs=["./logs.json"]);

------
### 博客
https://www.cnblogs.com/yangruiGB2312/p/9374377.html

In [4]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_score
from bayes_opt import BayesianOptimization

In [5]:
# 产生随机分类数据集，10个特征， 2个类别
x, y = make_classification(n_samples=1000,n_features=10,n_classes=2)

rf = RandomForestClassifier()
print(np.mean(cross_val_score(rf, x, y, cv=20, scoring='roc_auc')))

0.9491571794871796


我们先定义一个目标函数，里面放入我们希望优化的函数。比如此时，函数输入为随机森林的所有参数，输出为模型交叉验证5次的AUC均值，作为我们的目标函数。因为bayes_opt库只支持最大值，所以最后的输出如果是越小越好，那么需要在前面加上负号，以转为最大值。由于bayes优化只能优化连续超参数，因此要加上int()转为离散超参数。

In [7]:
def rf_cv(n_estimators, min_samples_split, max_features, max_depth):
    val = cross_val_score(
        RandomForestClassifier(n_estimators=int(n_estimators),
            min_samples_split=int(min_samples_split),
            max_features=min(max_features, 0.999), # float
            max_depth=int(max_depth),
            random_state=2
        ),
        x, y, 'roc_auc', cv=5
    ).mean()
    return val


In [9]:
 #里面的第一个参数是我们的优化目标函数，第二个参数是我们所需要输入的超参数名称，以及其范围。超参数名称必须和目标函数的输入名称一一对应。
rf_bo = BayesianOptimization(
        rf_cv,
        {'n_estimators': (10, 250),
        'min_samples_split': (2, 25),
        'max_features': (0.1, 0.999),
        'max_depth': (5, 15)}
    )

rf_bo.maximize()
print rf_bo.max
#  rf_bo.res

|   iter    |  target   | max_depth | max_fe... | min_sa... | n_esti... |
-------------------------------------------------------------------------
|  1        |  0.9561   |  13.85    |  0.3181   |  24.86    |  186.9    |
|  2        |  0.956    |  6.889    |  0.7467   |  22.9     |  107.1    |
|  3        |  0.953    |  11.62    |  0.6766   |  11.48    |  127.4    |
|  4        |  0.9525   |  13.74    |  0.1578   |  17.46    |  155.4    |
|  5        |  0.9551   |  12.44    |  0.4494   |  13.3     |  172.4    |
|  6        |  0.9561   |  7.295    |  0.7485   |  24.97    |  10.19    |
|  7        |  0.956    |  5.532    |  0.2249   |  24.6     |  10.03    |
|  8        |  0.9387   |  14.89    |  0.1327   |  2.334    |  10.24    |
|  9        |  0.9573   |  5.037    |  0.8789   |  3.447    |  249.5    |
|  10       |  0.9564   |  5.017    |  0.9548   |  24.72    |  198.1    |
|  11       |  0.9543   |  14.96    |  0.976    |  23.16    |  248.5    |
|  12       |  0.9575   |  5.145    | 

上面bayes算法得到的参数并不一定最优，当然我们会遇到一种情况，就是我们已经知道有一组或是几组参数是非常好的了，我们想知道其附近有没有更好的。这个操作相当于上文bayes优化中的Explore操作，而bayes_opt库给了我们实现此方法的函数：

In [37]:
###   已移除
'''
rf_bo.explore(
    {'n_estimators': [10, 100, 200],
        'min_samples_split': [2, 10, 20],
        'max_features': [0.1, 0.5, 0.9],
        'max_depth': [5, 10, 15]
    }
)
'''

"\nrf_bo.explore(\n    {'n_estimators': [10, 100, 200],\n        'min_samples_split': [2, 10, 20],\n        'max_features': [0.1, 0.5, 0.9],\n        'max_depth': [5, 10, 15]\n    }\n)\n"

设置核函数
https://scikit-learn.org/stable/modules/generated/sklearn.gaussian_process.GaussianProcessRegressor.html

In [35]:
gp_param={'kernel':None}
rf_bo.maximize(**gp_param)

|   iter    |  target   | max_depth | max_fe... | min_sa... | n_esti... |
-------------------------------------------------------------------------
|  66       |  0.9517   |  13.13    |  0.1642   |  12.18    |  109.0    |
|  67       |  0.9573   |  6.266    |  0.6128   |  14.73    |  15.6     |
|  68       |  0.9573   |  9.22     |  0.2138   |  19.12    |  157.5    |
|  69       |  0.9585   |  7.411    |  0.2246   |  16.21    |  177.8    |
|  70       |  0.959    |  5.431    |  0.5287   |  8.005    |  65.03    |
|  71       |  0.9562   |  5.756    |  0.9523   |  4.931    |  66.84    |
|  72       |  0.9496   |  5.0      |  0.1      |  10.15    |  67.78    |
|  73       |  0.9508   |  8.867    |  0.1863   |  8.421    |  64.02    |
|  74       |  0.9562   |  6.918    |  0.7782   |  15.5     |  181.3    |
|  75       |  0.9553   |  5.547    |  0.8654   |  7.909    |  61.37    |
|  76       |  0.9572   |  7.675    |  0.4324   |  12.64    |  178.6    |
|  77       |  0.9549   |  7.57     | 

In [36]:
rf = RandomForestClassifier(max_depth=6, max_features=0.39517, min_samples_split=2, n_estimators=250)
np.mean(cross_val_score(rf, x, y, cv=20, scoring='roc_auc'))

0.9594251282051282

-----
### 其他参考博客
http://www.sohu.com/a/165565029_465975
https://yq.aliyun.com/ziliao/313007